# Convolutional Neural Network (CNN) Exploration

In this notebook, we will lay out our approach for a CNN based on transfer learning experience

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import layers, models


model = VGG16(weights="imagenet", include_top=False, input_shape=X_train[0].shape)